RESTART kernel prior to running after any changes to config files...

### import configuration and functions modules

In [ ]:
%%time
import config as cf
import functions as f
from list_builder import prepare_master_list, build_list

### build pkl files
master, fur, sg(special group), active_each_month, order proposals, month percentage

In [ ]:
%%time
%run build_program_files

### build compensation tables
convert from input Excel file, detects config sample_mode if selected

In [ ]:
%%time
%run make_pay_tables_from_excel

### skeleton

In [ ]:
%%time
%run make_skeleton master

### standalone
standalone must be rerun after job level model change...

In [ ]:
%%time
if cf.compute_with_job_changes:
    %run standalone_with_job_changes prex
else:
    %run standalone_no_job_changes prex

### calculate for each list order
(including list conditions, job count changes, and possible furlough recall schedule)

In [ ]:
%%time
%run compute_measures p1 prex ratio

In [ ]:
%%time
%run compute_measures p2 prex count

In [ ]:
%%time
%run compute_measures p3 prex

### merge an ordered list with the master list

In [ ]:
%%time
%run join_inactives master p1 final ffill

In [ ]:
%%time
f.print_config_selections()

### cells below for demonstration only

### build a 'hybrid' list example:
(stored as 'dill/hybrid.pkl')

In [ ]:
%%time
# this will build a "hybrid" list with equal weighting given to longevity and standalone job position
master = prepare_master_list()
build_list(master, ['ldate', 'jobp'], [.5, .5])

### sample of files created

In [ ]:
%%time
# print only the first 5 lines of each file
file_names = ['master', 'skel', 'stand', 'active_each_month', 'p1', 'pay_table_enhanced_with_fur_indexed', 'ds1', 'hybrid', cf.case_study + '_final']
for file in file_names:
    df = pd.read_pickle('dill/' + file + '.pkl')
    print('< ' + file.upper() + ' >', '\n', 'row count: ' + str(len(df)),
        '\n', 'column count: '+ str(len(df.columns)), '\n',
        df.columns, '\n',
        '\n', df.head().to_string(line_width=120), '\n\n', '-' * 80, '\n')